In [1]:
import argparse
from datetime import datetime
import json
import logging
import os
import math
import sys
import time
import topologyqueries,linkedlist,update_topology,spanning_tree

from gridappsd import GridAPPSD, DifferenceBuilder, utils, topics as t
from gridappsd.topics import simulation_input_topic, simulation_log_topic, simulation_output_topic


#model_mrid = sim_request["power_system_config"]["Line_name"]
#model_mrid = "_C125761E-9C21-4CA9-9271-B168150DE276" #ieee13training
#model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #final9500node
#model_mrid = "_AAE94E4A-2465-6F5E-37B1-3E72183A4E44" #test9500
#model_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" #ieee13assets
#model_mrid="_C1C3E687-6FFD-C753-582B-632A27E28507" # IEEE 123
model_mrid="_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"

requestedTime=1570041125



global BaseConnDict,BaseTermDict,TermList, NodeList
global XfmrKeys,XfmrDict,SwitchKeys,SwitchDict,DG_query

os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'

gapps = GridAPPSD()
assert gapps.connected

In [2]:
NodeQuery=topologyqueries.node_query(gapps,model_mrid)

ConnNodeEquipDict = {}

StartTime = time.process_time()

for i0 in range(len(NodeQuery)):
    node=NodeQuery[i0]['cnid']['value']

    ConnNodeEquipDict[node] = {}
    ConnNodeEquipDict[node]['name'] = NodeQuery[i0]['busname']['value']
    ConnNodeEquipDict[node]['TopologicalNode'] = NodeQuery[i0]['tpnid']['value']
    
    ConnNodeEquipDict[node]['ACLineSegment'] = []
    ConnNodeEquipDict[node]['Breaker'] = []
    ConnNodeEquipDict[node]['EnergyConsumer'] = []
    ConnNodeEquipDict[node]['Fuse'] = []
    ConnNodeEquipDict[node]['LinearShuntCompensator'] = []
    ConnNodeEquipDict[node]['LoadBreakSwitch'] = []
    ConnNodeEquipDict[node]['PowerTransformer'] = []
    ConnNodeEquipDict[node]['RatioTapChanger'] = []
    ConnNodeEquipDict[node]['Recloser'] = []
    ConnNodeEquipDict[node]['TransformerEnd'] = []    
    ConnNodeEquipDict[node]['SynchronousMachine'] = []
    ConnNodeEquipDict[node]['PowerElectronicsConnection'] = []
    ConnNodeEquipDict[node]['measurements'] = []

print('Processed ', i0+1, 'nodes in ', time.process_time() - StartTime, "seconds")


Processed  22 nodes in  0.0003386799999999801 seconds


In [3]:
EquipDict = {}
EquipDict['ACLineSegment'] = {}
EquipDict['Breaker'] = {}
EquipDict['EnergyConsumer'] = {}
EquipDict['Fuse'] = {}
EquipDict['LinearShuntCompensator'] = {}
EquipDict['LoadBreakSwitch'] = {}
EquipDict['PowerTransformer'] = {}
EquipDict['RatioTapChanger'] = {}
EquipDict['Recloser'] = {}
EquipDict['TransformerTank'] = {}    
EquipDict['SynchronousMachine'] = {}
EquipDict['PowerElectronicsConnection'] = {}

In [4]:
MeasurementQuery=topologyqueries.measurement_query(gapps,model_mrid)

In [5]:
StartTime = time.process_time()

for i1 in range(len(MeasurementQuery)):    
    node = MeasurementQuery[i1]['cnid']['value']
    eqtype = MeasurementQuery[i1]['meastype']['value']
    eqid = MeasurementQuery[i1]['eqid']['value']
    
    ConnNodeEquipDict[node]['measurements'].append(MeasurementQuery[i1]['measid']['value'])
    if eqid not in ConnNodeEquipDict[node][eqtype]:
        ConnNodeEquipDict[node][eqtype].append(eqid)
        
    if eqid not in EquipDict[eqtype]: EquipDict[eqtype][eqid] = {}
        
    if 'node1' in EquipDict[eqtype][eqid]:
        if EquipDict[eqtype][eqid]['node1'] != node:
            EquipDict[eqtype][eqid]['node2'] = node
            EquipDict[eqtype][eqid]['term2'] = MeasurementQuery[i1]['trmid']['value']
    else:
        EquipDict[eqtype][eqid]['name'] = MeasurementQuery[i1]['eqname']['value']
        EquipDict[eqtype][eqid]['node1'] = node
        EquipDict[eqtype][eqid]['term1'] = MeasurementQuery[i1]['trmid']['value']
        
print('Processed ', i1+1, 'objects in ', time.process_time() - StartTime, "seconds")


Processed  183 objects in  0.0006835000000000035 seconds


In [6]:
LineQuery = topologyqueries.line_query(gapps, model_mrid)
[XfmrDict,XfmrKeys]=topologyqueries.transformer_query(gapps,model_mrid)

eqtype = 'ACLineSegment'

In [7]:
LineQuery = topologyqueries.line_query(gapps, model_mrid)

eqtype = 'ACLineSegment'

for i2 in range(len(LineQuery)):
    
    eqid = LineQuery[i2]['id']['value']
    EquipDict['ACLineSegment'][eqid]['term1'] = LineQuery[i2]['term1']['value']
    EquipDict['ACLineSegment'][eqid]['term2'] = LineQuery[i2]['term2']['value']
    EquipDict['ACLineSegment'][eqid]['node1'] = LineQuery[i2]['node1']['value']
    EquipDict['ACLineSegment'][eqid]['node2'] = LineQuery[i2]['node2']['value']

In [8]:
QueryXfmrs="""
    # list all the terminals connected to a TransformerEnd for CIMWriter
    PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX c:  <http://iec.ch/TC57/CIM100#>
    SELECT ?class ?eqid ?endid ?tname ?tid ?bus ?cnid ?tpid ?seq ?phs ?ratedu WHERE {
    VALUES ?fdrid {"%s"} 
    ?fdr c:IdentifiedObject.mRID ?fdrid.
    {?pxf c:Equipment.EquipmentContainer ?fdr.
    ?end c:PowerTransformerEnd.PowerTransformer ?pxf.
    ?end c:PowerTransformerEnd.ratedU ?ratedu.
    ?pxf c:IdentifiedObject.mRID ?eqid.
    }
    UNION
    {?tank c:Equipment.EquipmentContainer ?fdr.
    ?end c:TransformerTankEnd.TransformerTank ?tank.
    ?tank c:IdentifiedObject.mRID ?eqid.
    ?end c:TransformerTankEnd.phases ?ph.
    }
    ?end c:TransformerEnd.Terminal ?t.
    ?t c:Terminal.ConnectivityNode ?cn. 
    ?t c:IdentifiedObject.name ?tname.

    ?cn c:ConnectivityNode.TopologicalNode ?tp.
    ?cn c:IdentifiedObject.name ?bus.
    ?t c:ACDCTerminal.sequenceNumber ?seq.
    bind(strafter(str(?end),"#") as ?endid).
    bind(strafter(str(?t),"#") as ?tid).
    bind(strafter(str(?cn),"#") as ?cnid).
    bind(strafter(str(?tp),"#") as ?tpid).
    bind(strbefore(str(?endclass), "E") as ?class)
    bind(strafter(str(?ph),"e.") as ?phs).
    ?end a ?classraw.
    bind(strafter(str(?classraw),"CIM100#") as ?endclass)
    }
    ORDER by ?class ?eqid ?tname ?endid ?bus ?cnid ?tpid ?seq ?phs ?ratedu
    """%model_mrid
results = gapps.query_data(query = QueryXfmrs, timeout = 60)
XfmrQuery = results['data']['results']['bindings']

In [9]:
# Build dictionary of FROM-TO nodes for all transformers
for i2 in range(len(XfmrQuery)):
    eqtype = XfmrQuery[i2]['class']['value']
    eqid=XfmrQuery[i2]['eqid']['value']
    seq=XfmrQuery[i2]['seq']['value']
    
    if eqid not in EquipDict[eqtype]:
        EquipDict[eqtype][eqid] = {}


    # Identify terminal sequence and create keys for new terminals
    if seq == '1' or seq == 1:
        # Primary winding
        EquipDict[eqtype][eqid]['bus1']=XfmrQuery[i2]['bus']['value']
        EquipDict[eqtype][eqid]['term1']=XfmrQuery[i2]['tid']['value']
        EquipDict[eqtype][eqid]['node1']=XfmrQuery[i2]['cnid']['value']
        EquipDict[eqtype][eqid]['tpnode1']=XfmrQuery[i2]['tpid']['value']
        EquipDict[eqtype][eqid]['tname1']=XfmrQuery[i2]['tname']['value']
        
        if 'ratedu' in XfmrQuery[i2]: EquipDict[eqtype][eqid]['volt1']=int(float(XfmrQuery[i2]['ratedu']['value']))
        else: EquipDict[eqtype][eqid]['volt1']=0
        if 'phs' in XfmrQuery[i2]: EquipDict[eqtype][eqid]['phase1']=XfmrQuery[i2]['phs']['value'] 
        else: EquipDict[eqtype][eqid]['phase1']={}

    elif seq == '2' or seq == 2:
        EquipDict[eqtype][eqid]['bus2']=XfmrQuery[i2]['bus']['value']
        EquipDict[eqtype][eqid]['term2']=XfmrQuery[i2]['tid']['value']
        EquipDict[eqtype][eqid]['node2']=XfmrQuery[i2]['cnid']['value']
        EquipDict[eqtype][eqid]['tpnode2']=XfmrQuery[i2]['tpid']['value']
        EquipDict[eqtype][eqid]['tname2']=XfmrQuery[i2]['tname']['value']
        
        if 'ratedu' in XfmrQuery[i2]: EquipDict[eqtype][eqid]['volt2']=int(float(XfmrQuery[i2]['ratedu']['value']))
        else: EquipDict[eqtype][eqid]['volt2']=0
        if 'phs' in XfmrQuery[i2]: EquipDict[eqtype][eqid]['phase2']=XfmrQuery[i2]['phs']['value'] 
        else: EquipDict[eqtype][eqid]['phase2']={}

    elif seq == '3' or seq == 3:
        EquipDict[eqtype][eqid]['bus3']=XfmrQuery[i2]['bus']['value']
        EquipDict[eqtype][eqid]['term3']=XfmrQuery[i2]['tid']['value']
        EquipDict[eqtype][eqid]['node3']=XfmrQuery[i2]['cnid']['value']
        EquipDict[eqtype][eqid]['tpnode3']=XfmrQuery[i2]['tpid']['value']
        EquipDict[eqtype][eqid]['tname3']=XfmrQuery[i2]['tname']['value']
        
        if 'ratedu' in XfmrQuery[i2]: EquipDict[eqtype][eqid]['volt3']=int(float(XfmrQuery[i2]['ratedu']['value']))
        else: EquipDict[eqtype][eqid]['volt3']=0
        if 'phs' in XfmrQuery[i2]: EquipDict[eqtype][eqid]['phase3']=XfmrQuery[i2]['phs']['value'] 
        else: EquipDict[eqtype][eqid]['phase3']={}
    else:
        raise ValueError('Unsupported transformer sequence value ', seq)

In [10]:
i2

13

In [11]:
XfmrQuery[i2]

{'class': {'type': 'literal', 'value': 'TransformerTank'},
 'eqid': {'type': 'literal', 'value': '_E2E0FC64-8D45-4C55-BDB9-EAB827A46FBC'},
 'endid': {'type': 'literal',
  'value': '_1B9E29BF-7D12-45B0-965A-CA4071F66E3C'},
 'tname': {'type': 'literal', 'value': 'reg3_T2'},
 'tid': {'type': 'literal', 'value': '_F5258B9A-B51E-40DD-9A06-5918E41F3C35'},
 'bus': {'type': 'literal', 'value': 'rg60'},
 'cnid': {'type': 'literal', 'value': '_673E896A-DCBF-4E43-9924-BEB31C5B6005'},
 'tpid': {'type': 'literal', 'value': '_54D52F71-9CC6-4C4D-8D5F-75A1C8F23BD8'},
 'seq': {'type': 'literal', 'value': '2'},
 'phs': {'type': 'literal', 'value': 'C'}}

In [12]:
index=0
TerminalsDict = {}
NodeList = []
TermList = []

StartTime = time.process_time()

eqtype = 'ACLineSegment'

SwitchKeys = list(EquipDict['Breaker'].keys()) + list(EquipDict['Fuse'].keys()) + list(EquipDict['LoadBreakSwitch'].keys()) + list(EquipDict['Recloser'].keys())



In [13]:
StartTime = time.process_time()

eqtype = 'PowerTransformer'

EquipKeys = list(EquipDict[eqtype])

for i1 in range(len(EquipKeys)):

    term1=EquipDict[eqtype][EquipKeys[i1]]['term1']
    term2=EquipDict[eqtype][EquipKeys[i1]]['term2']
    node1=EquipDict[eqtype][EquipKeys[i1]]['node1']
    node2=EquipDict[eqtype][EquipKeys[i1]]['node2']

    # Create keys for new terminals
    TerminalsDict[term1] = {}
    TerminalsDict[term2] = {}
    TerminalsDict[term1]['term'] = 2*i1+1
    TerminalsDict[term2]['term'] = 2*i1+2
    TermList.append(term1)
    TermList.append(term2)

    # If node1 or node2 not in dict, create new keys
    if 'node' not in ConnNodeEquipDict[node1]:
        ConnNodeEquipDict[node1]['node'] = index+1
        ConnNodeEquipDict[node1]['list'] = 0
        index = index+1
        NodeList.append(node1)

    if 'node' not in ConnNodeEquipDict[node2]: 
        ConnNodeEquipDict[node2]['node'] = index+1
        ConnNodeEquipDict[node2]['list'] = 0
        index = index+1
        NodeList.append(node2)

    # 1. Move node list variables to terinal next    
    TerminalsDict[term1]['next'] = ConnNodeEquipDict[node1]['list']
    TerminalsDict[term2]['next'] = ConnNodeEquipDict[node2]['list']

    # 2. Populate Terminal list far field with nodes
    TerminalsDict[term1]['far'] = ConnNodeEquipDict[node2]['node']
    TerminalsDict[term2]['far'] = ConnNodeEquipDict[node1]['node']

    # 3. Populate Connectivity nodes list with terminals
    ConnNodeEquipDict[node1]['list'] = TerminalsDict[term1]['term']
    ConnNodeEquipDict[node2]['list'] = TerminalsDict[term2]['term']


print("Processed ", i1+1, eqtype, "objects in ", time.process_time() - StartTime, "seconds")


Processed  2 PowerTransformer objects in  0.0003520399999999979 seconds


In [14]:
BaseConnDict = json.dumps(ConnNodeEquipDict)
BaseTermDict = json.dumps(TerminalsDict)

In [15]:
[TerminalsDict, ConnNodeDict]=update_topology.topology_update(BaseConnDict,BaseTermDict,EquipDict['LoadBreakSwitch'],SwitchKeys,TermList)

SubstationRoots = []
# Iterate through all substation transformers
for i6 in range(len(XfmrKeys)):

    # Identify if distribution substation transformer
    if ((int(XfmrDict[XfmrKeys[i6]]['volt1']) >= 34000 and 34000 >= int(XfmrDict[XfmrKeys[i6]]['volt2']) > 1000) or 
        (int(XfmrDict[XfmrKeys[i6]]['volt2']) >= 34000 and 34000 >= int(XfmrDict[XfmrKeys[i6]]['volt1']) > 1000)):
        SubstationRoots.append(XfmrKeys[i6])
        
[SubTree, ConnNodeDict] = SpanningTree(ConnNodeDict, TerminalsDict, NodeList, XfmrDict, SubstationRoots, 'single')

KeyError: '_52DE9189-20DC-4C73-BDEE-E960FE1F9493'

In [ ]:
EquipDict['LoadBreakSwitch'][SwitchKeys[0]]

In [ ]:
SwitchKeys[0]

In [ ]:
EquipDict['TransformerTank']

In [ ]:
EquipDict['PowerTransformer']

In [ ]:
ConnNodeEquipDict['_0018BD86-235A-4FF5-AA48-FF9AAE39A5DC']

In [16]:
from gridappsd import topics as t
topic = t.REQUEST_POWERGRID_DATA

message = {
    "requestType": "QUERY_OBJECT_DICT",
    "modelId": model_mrid,
    "objectId": '_43EF8365-F932-409B-A51E-FBED3F6DFFAA',
    "resultFormat": "JSON"
}

gapps.get_response(topic, message)

{'data': [{'id': '_43EF8365-F932-409B-A51E-FBED3F6DFFAA',
   'ConductingEquipment.BaseVoltage': '_2A158E0C-CD01-4A50-AEBA-59D761FCF15D',
   'Equipment.EquipmentContainer': '_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62',
   'IdentifiedObject.mRID': '_43EF8365-F932-409B-A51E-FBED3F6DFFAA',
   'IdentifiedObject.name': 'fuse1',
   'PowerSystemResource.Location': '_9702BD06-90C3-40B5-9311-47FA1C2A9811',
   'Switch.normalOpen': 'false',
   'Switch.open': 'false',
   'Switch.ratedCurrent': '65',
   'Switch.retained': 'true',
   'type': 'Fuse'}],
 'responseComplete': True,
 'id': '377179489'}

In [ ]:
SwitchKeys = list(EquipDict['Breaker'].keys()) + list(EquipDict['Fuse'].keys()) + list(EquipDict['LoadBreakSwitch'].keys()) + list(EquipDict['Recloser'].keys())
SwitchKeys

In [18]:
EquipDict.keys()

dict_keys(['ACLineSegment', 'Breaker', 'EnergyConsumer', 'Fuse', 'LinearShuntCompensator', 'LoadBreakSwitch', 'PowerTransformer', 'RatioTapChanger', 'Recloser', 'TransformerTank', 'SynchronousMachine', 'PowerElectronicsConnection'])